In [1]:
%load_ext autoreload
%autoreload 2
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import matplotlib.pyplot as plt
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np


In [2]:
from shapely import geometry as gmty

In [3]:
from geofeather import to_geofeather, from_geofeather

In [4]:
shapefile = "/home/mlopez/EXEC/Shapefiles/territoire_guide.shp"

In [5]:
#path for 11 models
path = "/home/mlopez/EXEC/Tg_annual_11_models/"

# Convert all lat lon of 11 models into points 

In [6]:
# It takes 20s
%time dfpoints = pcdf.data_to_points(path)

CPU times: user 20.8 s, sys: 1.55 s, total: 22.4 s
Wall time: 8.31 s


In [7]:
dfpoints.head()

,lat,lon,geometry
0,66.623306,-89.045212,POINT (-89.04521 66.62331)
1,66.623306,-88.961884,POINT (-88.96188 66.62331)
2,66.623306,-88.878548,POINT (-88.87855 66.62331)
3,66.623306,-88.795219,POINT (-88.79522 66.62331)
4,66.623306,-88.711891,POINT (-88.71189 66.62331)


# Create polygons

In [8]:
#takes 8s
%time dfpoints.geometry  = pcdf.process_points_poly(dfpoints)

CPU times: user 8.51 s, sys: 336 ms, total: 8.84 s
Wall time: 8.83 s


In [9]:
dfpoints.head()

,lat,lon,geometry
0,66.623306,-89.045212,"POLYGON ((-89.08688 66.58164, -89.00355 66.581..."
1,66.623306,-88.961884,"POLYGON ((-89.00355 66.58164, -88.92022 66.581..."
2,66.623306,-88.878548,"POLYGON ((-88.92021 66.58164, -88.83688 66.581..."
3,66.623306,-88.795219,"POLYGON ((-88.83688 66.58164, -88.75356 66.581..."
4,66.623306,-88.711891,"POLYGON ((-88.75356 66.58164, -88.67023 66.581..."


In [10]:
### TAKES 10 min for all the regions TG
%time  dfpolyshape = pcdf.make_grid(dfpoints, shapefile) #10 min with overlay #1miin with sjoin

CPU times: user 9min 23s, sys: 10.1 s, total: 9min 33s
Wall time: 9min 7s


In [11]:
dfpolyshape.head()

,lat,lon,TER_GUIDE,geometry
0,52.957191,-67.712730,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
1,52.957191,-67.629402,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."
2,52.957191,-67.546066,6opqr,"POLYGON ((-67.56730 52.91553, -67.57974 52.915..."
3,52.873859,-67.796059,6opqr,"POLYGON ((-67.75439 52.89329, -67.75439 52.832..."
4,52.873859,-67.712730,6opqr,"POLYGON ((-67.75439 52.83219, -67.75439 52.893..."


In [12]:
to_geofeather(dfpolyshape, 'Grid-TerritoiresGuides.feather')

In [13]:
#For one region
#OneA =  dfpolyshape[dfpolyshape.TER_GUIDE == "1a"]
#OneA

# Create time periods to merge grid with data

In [25]:
# MAKE FOR HERE

## Load data into a dataframe, convert Temperature into Celsius and get tuple of 3 periods

In [14]:
# Takes 1min 30 s - Creates a tuple of 3 df, each with every period
%time Ttg_m = pcdf.dft_map_models_periods("/home/mlopez/EXEC/Processed Data Varibales/NorESM1-M_rcp45_tg_mean_annual.nc", "tg_mean")

/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds['time'] = index.to_datetimeindex()


CPU times: user 1min 16s, sys: 1min 39s, total: 2min 56s
Wall time: 7.67 s


## Procesing periods to merge with data

### Historic Period

In [15]:
dfh2 = pcdf.proc_period(Ttg_m, 0)

In [16]:
dfh2

,index,time,lat,lon,tg_mean
0,0,0.0,40.041039,-89.045212,11.474038
1,1,0.0,40.041039,-88.961884,11.452959
2,2,0.0,40.041039,-88.878548,11.440704
3,3,0.0,40.041039,-88.795219,11.419402
4,4,0.0,40.041039,-88.711891,11.434692
...,...,...,...,...,...
76201,76201,0.0,66.623306,-61.629639,-9.656209
76202,76202,0.0,66.623306,-61.546310,-9.948932
76203,76203,0.0,66.623306,-61.462978,-9.735876
76204,76204,0.0,66.623306,-61.379650,-9.655593


In [17]:
#For one subregion
#OneA.set_index(["lat","lon"])

In [18]:
#For all subregions
dfpolyshape.set_index(["lat","lon"])

TER_GUIDE  \
lat       lon                    
52.957191 -67.712730     6opqr   
          -67.629402     6opqr   
          -67.546066     6opqr   
52.873859 -67.796059     6opqr   
          -67.712730     6opqr   
...                        ...   
45.041668 -72.208336        2c   
          -72.125000        2c   
          -72.041672        2c   
          -71.958336        2c   
          -71.875000        2c   

                                                               geometry  
lat       lon                                                            
52.957191 -67.712730  POLYGON ((-67.67107 52.93424, -67.67107 52.915...  
          -67.629402  POLYGON ((-67.67107 52.91553, -67.67107 52.934...  
          -67.546066  POLYGON ((-67.56730 52.91553, -67.57974 52.915...  
52.873859 -67.796059  POLYGON ((-67.75439 52.89329, -67.75439 52.832...  
          -67.712730  POLYGON ((-67.75439 52.83219, -67.75439 52.893...  
...                                                                 ...  
45.041668 -72.208336  POLYGON ((-72.25000 45.00450, -72.25000 45.083...  
          -72.125000  POLYGON ((-72.16666 45.00508, -72.16666 45.083...  
          -72.041672  POLYGON ((-72.08334 45.00564, -72.08334 45.083...  
          -71.958336  POLYGON ((-72.00000 45.00667, -72.00000 45.083...  
          -71.875000  POLYGON ((-71.91666 45.04812, -71.91666 45.083...  

[33198 rows x 2 columns]

## Merge data with mask 

In [19]:
#for one region
#dfTG2 = pd.merge(df_h, OneA, on=["lat","lon"])

In [20]:
dfTGall = pd.merge(dfh2, dfpolyshape, on=["lat","lon"])

In [ ]:
#CONCAT DF HERE

In [21]:
dfTGall

,index,time,lat,lon,tg_mean,TER_GUIDE,geometry
0,12089,0.0,44.957512,-74.629120,7.079183,1a,"POLYGON ((-74.62590 44.99918, -74.58746 44.999..."
1,12090,0.0,44.957512,-74.545792,7.053729,1a,"POLYGON ((-74.58746 44.99876, -74.58746 44.999..."
2,12091,0.0,44.957512,-74.462463,7.030324,1a,"POLYGON ((-74.50413 44.99784, -74.50413 44.999..."
3,12092,0.0,44.957512,-74.379128,7.015834,1a,"POLYGON ((-74.42079 44.99530, -74.42079 44.999..."
4,12093,0.0,44.957512,-74.295799,6.978581,1a,"POLYGON ((-74.33746 44.99215, -74.33746 44.999..."
...,...,...,...,...,...,...,...
16563,43761,0.0,52.873859,-67.629402,-3.216939,6opqr,"POLYGON ((-67.67107 52.83219, -67.67107 52.915..."
16564,43762,0.0,52.873859,-67.546066,-3.166103,6opqr,"POLYGON ((-67.58773 52.83219, -67.58773 52.904..."
16565,44137,0.0,52.957191,-67.712730,-3.351753,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
16566,44138,0.0,52.957191,-67.629402,-3.251923,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."


In [27]:
dfTGall.TER_GUIDE.value_counts()

6cdefg    2359
6opqr     1609
6ab       1420
5bcd      1231
4bc       1057
6hi       1028
6j         950
6mn        923
3ab        652
5a         552
6kl        507
5ef        401
4f         389
2b         388
3d         367
3c         356
4de        356
5g         347
5hi        342
1a         338
4gh        304
5jk        244
2c         166
2a         143
4a         139
Name: TER_GUIDE, dtype: int64

In [28]:
TG4a = dfTGall[dfTGall.TER_GUIDE == "4a"]
TG4a

,index,time,lat,lon,tg_mean,TER_GUIDE,geometry
2623,19752,0.0,47.124088,-79.462257,3.518152,4a,"POLYGON ((-79.44652 47.16575, -79.42059 47.165..."
2625,19753,0.0,47.124088,-79.378929,3.375563,4a,"POLYGON ((-79.42059 47.15410, -79.42059 47.165..."
2627,19754,0.0,47.124088,-79.295601,3.180336,4a,"POLYGON ((-79.33237 47.16575, -79.29047 47.165..."
2771,20016,0.0,47.207420,-79.462257,3.351269,4a,"POLYGON ((-79.45147 47.24909, -79.42059 47.249..."
2772,20017,0.0,47.207420,-79.378929,3.388705,4a,"POLYGON ((-79.42059 47.16575, -79.42059 47.249..."
...,...,...,...,...,...,...,...
4596,23399,0.0,48.124050,-79.045609,2.001026,4a,"POLYGON ((-79.08727 48.08238, -79.08727 48.165..."
4598,23400,0.0,48.124050,-78.962280,2.021969,4a,"POLYGON ((-79.00394 48.08238, -79.00394 48.165..."
4813,23738,0.0,48.207378,-79.128937,1.911239,4a,"POLYGON ((-79.08727 48.17824, -79.08727 48.165..."
4815,23739,0.0,48.207378,-79.045609,1.966652,4a,"POLYGON ((-79.08727 48.16571, -79.08727 48.178..."


In [29]:
TG4a.to_csv("/home/mlopez/EXEC/Tg_csv/TG4aNor.csv")

In [26]:
dfTGall.to_csv("/home/mlopez/EXEC/Tg_csv/NorESM1-M_rcp45_tg_mean_annual.csv")

# ---- Create GeoJson file ----

In [22]:
geometry = dfTGall["geometry"]

In [23]:
crs = {'init': "epsg:4326"}
gdf = GeoDataFrame(dfTGall, crs=crs, geometry=geometry)

In [24]:
gdf.to_file("TGall.json", driver="GeoJSON")